In [12]:
train = pd.read_csv("../dataset/train.csv")

### Set Weekday as Number

In [13]:
wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}
# train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])

### Recognizing Return

In [14]:
def add_return(data):
    """
    Add new feature "return" if ScanCount < 0.
    1: return occured, 0: no return.
    """
    data.loc[data.ScanCount < 0, "Return"] = 1
    data.loc[data.Return != 1, "Return"] = 0

### 각 DepartmentDescriptoin 당 ScanCount

In [15]:
# 주석 처리한 부분을 실행하면 0, 1로 변경되어서 실행 안하도록 코드 수정했습니다.

def count_per_desc(data):
    """
    OneHotEncoding DepartmentDescription 
    then multiply ScanCount.
    """
    dummies = pd.get_dummies(data.DepartmentDescription)
#     data[dummies.columns] = dummies
#     data_dummies = data.loc[:, "1-HR PHOTO":"WIRELESS"]
    data_dummies = dummies.apply(lambda x: x * data["ScanCount"])
    data_dummies = data_dummies.replace(-0, 0)
    data[dummies.columns] = data_dummies

### Grouping Data

In [16]:
def group_by_vn(data):
    """
    grouping data by VisitNumber.
    """
    grouped = data.groupby("VisitNumber")

    pass

### Category Counts

In [17]:
# return은 고려하지 않고 있는데, 이 부분에 대해서 논의가 필요할 듯 합니다.
# 원 코드 iloc --> loc로 컬럼 직관적으로 파악할 수 있도록 변경

def category_counts(data):
    """
    Count total number of unique DepartmentDescription made on each trip.
    """
    counts = []
    for array in np.asarray(data.loc[:, "1-HR PHOTO":"WIRELESS"]):
        count = 0
        for item in array:
            if item > 0:
                count += 1
        counts.append(count)
    cat_counts = pd.DataFrame(counts)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(7, "CategoryCounts", cat_counts)
    return data

### Using Most Frequent Fineline Numbers (OneHotEncoding)

In [18]:
def fineline_dummies(data):
    """
    OneHotEncoding FinelineNumber which appear 
    more than 500 times each Trip Type.
    """
    fineline_ranks = data.FinelineNumber.value_counts()
    fineline_ranks = fineline_ranks[fineline_ranks > 500]
#     columns1 = list(fineline_ranks.index)
    dummies = pd.get_dummies(data.FinelineNumber)
    dummies = dummies[fineline_ranks.index]
    
    data_visit = original_data[["VisitNumber"]]
    dummy_data = pd.concat([dummies, data_visit], axis=1)
    dummy_data = dummy_data.groupby("VisitNumber").aggregate(np.sum) # agg("sum")보다 더 나은 성능
    return dummy_data

In [ ]:
# Grouping Data 이후에 join 실시
# tmp = train.join(dummy_data)

In [ ]:
# def add_category_counts_test(data):
#     alist = []
#     for array in np.asarray(data.iloc[:,3:]):
#         count = 0
#         for item in array:
#             if item > 0:
#                 count += 1
#         alist.append(count)
#     cat_counts = pd.DataFrame(alist)
#     cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
#     cat_counts = cat_counts.set_index(data.index)
#     data.insert(3, 'CategoryCounts', cat_counts)
#     return data